In [5]:
import os
import numpy as np
import cv2
import tqdm
import seaborn as sns
import matplotlib.pylab as plt
from keras.utils import to_categorical


In [6]:

def collect_examples(images_path: str) -> [[]]:
    """
    :param images_path: Path to data main directory. Should have 2 dir levels - each game
    should have images in its own dir
    :return: filename, source name, list with board points
    """
    all_examples = []
    all_sources = []

    dirs = [os.path.join(images_path, d) for d in os.listdir(images_path)
            if os.path.isdir(os.path.join(images_path, d))]

    for d in dirs:
        files = [os.path.join(d, f) for f in os.listdir(d) if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.JPG')]
        files.sort()

        for f in files:
            all_examples.append(f)
            all_sources.append(d)

    return np.array(all_examples), np.array(all_sources)


In [7]:
import matplotlib
# Specifying the backend to be used before importing pyplot
# to avoid "RuntimeError: Invalid DISPLAY variable"
matplotlib.use('agg')
import matplotlib.pyplot as plt
import keras
import numpy as np

class TrainingPlot(keras.callbacks.Callback):

    # This function is called when the training begins
    def on_train_begin(self, logs={}):
        # Initialize the lists for holding the logs, losses and accuracies
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    # This function is called at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):

        # Append the logs, losses and accuracies to the lists
        self.logs.append(logs)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))

        # Before plotting ensure at least 2 epochs have passed
        if len(self.losses) > 1:

            N = np.arange(0, len(self.losses))

            # You can chose the style of your preference
            # print(plt.style.available) to see the available options
            #plt.style.use("seaborn")

            # Plot train loss, train acc, val loss and val acc against epochs passed
            plt.figure()
            plt.plot(N, self.losses, label = "train_loss")
            plt.plot(N, self.acc, label = "train_acc")
            plt.plot(N, self.val_losses, label = "val_loss")
            plt.plot(N, self.val_acc, label = "val_acc")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(epoch))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            # Make sure there exists a folder called output in the current directory
            # or replace 'output' with whatever direcory you want to put in the plots
            plt.savefig('output/Epoch-{}.png'.format(epoch))
            plt.close()

In [8]:
root = '../../../data/dataset_yolo/'
examples = collect_examples(root)

In [9]:
data = np.zeros((len(examples[0]),66,66,3), dtype=np.float32)
labels = np.zeros((len(examples[0],)), dtype=np.int8)

# for i, example in enumerate(tqdm.tqdm(examples[0])):
for i, example in enumerate(examples[0]):
    image = cv2.imread(example)
    data[i] = image
    with open(example[:-4] + '.txt') as f:
        cl = int(f.read())
    labels[i] = cl
    if i == 20000:
        break
    

In [10]:
labels = to_categorical(labels)

In [11]:
x_train = data[:19000]
x_test = data[19000:]
y_train = labels[:19000]
y_test = labels[19000:]

In [12]:
img_wh = 66
num_channels = 3
input_shape = (img_wh, img_wh, num_channels)
num_classes = labels.shape[1]
print("Num classes: {}".format(num_classes))

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], img_wh, img_wh, num_channels)
x_test = x_test.reshape(x_test.shape[0], img_wh, img_wh, num_channels)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

Num classes: 3
x_train shape: (19000, 66, 66, 3)
Number of images in x_train 19000
Number of images in x_test 235505


In [13]:
sns.countplot(np.argmax(y_train, axis=1))


In [14]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(np.argmax(y_train, axis=1)),
                                                 np.argmax(y_train, axis=1))

In [19]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.optimizers import Adam
import tensorflow as tf

def base_model(img_w, img_h, input_shape, num_classes):

    model = Sequential()
    model.add(Conv2D(img_w, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Conv2D(img_w,(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(img_w*2, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(img_w*2, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

def base_model(img_w, img_h, input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(img_wh, kernel_size=(3,3), input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(Dense(128, activation=tf.nn.relu))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation=tf.nn.softmax))
    return model

model = base_model(img_wh, img_wh, input_shape, num_classes)
model.summary()

optimizer = Adam(lr=0.001)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 66)        1848      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 66)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               8650880   
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 387       
Total params: 8,653,115
Trainable params: 8,653,115
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
plot_losses = TrainingPlot()
history = model.fit(x_train, y_train, batch_size=128, epochs=3, verbose=False, validation_data=(x_test, y_test), callbacks=[plot_losses])
# class_weight=class_weights

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
loss, accuracy  = model.evaluate(x_test, y_test, verbose=False)

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show()

print(f'Test loss: {loss:.3}')
print(f'Test accuracy: {accuracy:.3}')